# 1. Import library


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
root = '/gdrive/My Drive'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
import torch
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from PIL import Image
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg
import math

import torchvision
from torchvision import models
import torchvision.transforms as T

from matplotlib.path import Path
import matplotlib.patches as patches

# 2. load pretrained models


2.1. model for detect wearing mask vs don't wearing mask

In [38]:
new_model=tf.keras.models.load_model('/gdrive/My Drive/Code/face_mask_detection_CNN/saved_new_model')

new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 86, 116, 32)       2432      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 43, 58, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 39, 54, 128)       102528    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 19, 27, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 65664)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 65664)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

2.2. model for detect incorrect mask vs no mask

In [ ]:
new_model2=tf.keras.models.load_model('/gdrive/My Drive/Code/face_mask_detection_CNN/class2_model')

new_model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 86, 116, 32)       2432      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 43, 58, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 39, 54, 128)       102528    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 19, 27, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 65664)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 65664)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

# 3. Capture test video by frames

In [ ]:
def video2frame(invideofilename, save_path):
    vidcap = cv2.VideoCapture(invideofilename)
    count = 0
    while True:
      success,image = vidcap.read()
      if not success:
          break
      print ('Read a new frame: ', success)
      fname = "{}.jpg".format("{0:05d}".format(count))
      cv2.imwrite(save_path + fname, image) # save frame as JPEG file
      count += 1
    print("{} images are extracted in {}.". format(count, save_path))

invideofilename = '/gdrive/My Drive/demo2.mp4'
save_path = '/gdrive/My Drive/demo_capture2/'
video2frame(invideofilename,save_path)

0 images are extracted in /gdrive/My Drive/demo_capture2/.


# 4. Define various functions for processing the images



In [ ]:
# convert (r, g, b) to color hexcode
def rgb_to_hex(tuple):
    r, g, b = int(tuple[0]), int(tuple[1]), int(tuple[2])
    return '#' + hex(r)[2:].zfill(2) + hex(g)[2:].zfill(2) + hex(b)[2:].zfill(2)

In [ ]:
#Download Model : torchvision.keypoint rcnn model 사용
keypoint_model = models.detection.keypointrcnn_resnet50_fpn(pretrained=True).eval()

def extract_clothes_color(model, img):
    #Image to Tensor
    trf = T.Compose([
        T.ToTensor()
    ])
    
    input_img = trf(img)
    THRESHOLD = 0.95
    
    # cropped_list : list for saving human detection information
    cropped_list = []
    out = model([input_img])[0]

    codes = [
        Path.MOVETO,
        Path.LINETO,
        Path.LINETO
    ]


    # box : list of x, y coordinates of human detected box
    # score : probability of detection to be human
    # keypoints : list of body keypoints
    for box, score, keypoints in zip(out['boxes'], out['scores'], out['keypoints']):
        score = score.detach().numpy()
        if score < THRESHOLD:
            continue
        box = box.detach().numpy()
        keypoints = keypoints.detach().numpy()[:, :2]
        # extract left shoulder color
        left_x = keypoints[5:10:2][0][0].astype(np.int64).item()
        left_y = keypoints[5:10:2][0][1].astype(np.int64).item()
        left_r,left_g,left_b = img.getpixel((left_x, left_y))

        # extract right shoulder color
        right_x = keypoints[6:11:2][0][0].astype(np.int64).item()
        right_y = keypoints[6:11:2][0][1].astype(np.int64).item()
        right_r, right_g, right_b = img.getpixel((right_x, right_y))
        
        # cropped_img = crop original image with human detection box
        cropped_img = np.array(img)
        cropped_img = cropped_img[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
        xy = (int(box[0]), int(box[1]))

        # color : get average color of left, right shoulder
        color = (int((left_r+right_r)/2), int((left_g+right_g)/2), int((left_b+right_b)/2))
        cropped_list.append((cropped_img, xy, color))
    
    return cropped_list

In [ ]:
def extract_mask(img): 
    if img is None:
        return []

    # load pre-trained Haar cascade model in OpenCV for detect eye point
    face_cascade = cv2.CascadeClassifier('/gdrive/My Drive/Code/face_preprocessing/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('/gdrive/My Drive/Code/face_preprocessing/haarcascade_eye.xml')

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 3)

    # Append each eye point in the detected face
    eye_point = []
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        
        eyes = eye_cascade.detectMultiScale(roi_gray)
        eyes = eyes[:2]
        for (ex,ey,ew,eh) in eyes:
            xx = ex + ew // 2
            yy = ey + eh // 2
            eye_point.append((x+xx, y+yy))
            r = (ew + eh) // 4

    # Return empty list if there are no detected eyes
    if len(eye_point) < 2:
        return []

    # Set the left and right eyes by comparing the location
    eye1 = eye_point[0]
    eye2 = eye_point[1]
    length = math.sqrt(((eye1[0] - eye2[0]) ** 2) + ((eye1[1] - eye2[1]) ** 2))

    if eye1[0] > eye2[0]:
        right = eye1
        left = eye2
    elif eye1[0] < eye2[0]:
        right = eye2
        left = eye1

    # Rotated the image to fit the eye level
    if left[1] > right[1]:
        angle = math.atan2((right[1]-left[1]), (right[0]-left[0]))
        theta = angle*(180/math.pi);
        rotated = imutils.rotate(img,theta)
    elif left[1] <= right[1]:
        angle = math.atan2((left[1]-right[1]), (right[0]-left[0]))
        theta = angle*(180/math.pi);
        rotated = imutils.rotate(img,-theta)

    # Crop the rotated image by estimated size face
    cropped = rotated[int(left[1] - 0.6 * length):int(left[1] + 1.8 * length), int(left[0] - 0.6 * length):int(right[0] + 0.6 * length)]
    
    if cropped.size == 0:
        return []
    
    # Resize the cropped image by mask region
    resized = cv2.resize(cropped, dsize = (120, 140), interpolation=cv2.INTER_AREA)
    mask_region = resized[50:, :]

    # Return the mask region image and the location of the face region
    return mask_region, (int(left[0] - 0.6 * length), int(left[1] - 0.6 * length), int(left[0] + 1.6 * length), int(left[1] + 1.8 * length))

In [ ]:
!pip install webcolors
import webcolors
from webcolors import CSS21_HEX_TO_NAMES

In [ ]:
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS21_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        color_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        color_name = closest_colour(requested_colour)
    return color_name

# 5. Test our model for captured images and save the results

In [ ]:
for i in range(504):
    # read frames of video.
    img_string = '/gdrive/My Drive/output/mp4_capture/{0:05d}.jpg'.format(i)
    img_org = cv2.imread(img_string)

    print("{0:05d}:".format(i), end = " ")
    try:
        pil_img = Image.fromarray(img_org)
        # get list of information of each person
        cropped_list = extract_clothes_color(keypoint_model, pil_img)
        for (cropped_img, xy, color) in cropped_list:
            color_name = get_colour_name(color)
            print ("color name:", color_name)
            face_img, location = extract_mask(cropped_img)
            face_img = (np.expand_dims(face_img, 0))
            predictions_single=new_model.predict(face_img)
            correctness = np.argmax(predictions_single[0])
            x,y,x2,y2 = location

            # when a person doesn't correctly wear mask
            if correctness == 0:
                predictions_mask_incor = new_model2.predict(face_img)
                correctness_mask_incor = np.argmax(predictions_mask_incor[0])
                # when a person doesn't have mask
                if correctness_mask_incor == 0:
                    cv2.rectangle(img_org, (x + xy[0], y + xy[1]), (x2 + xy[0], y2 + xy[1]), (0, 0, 255), 4)
                    cv2.putText(img_org, "No mask", (x + xy[0], y - 10 + xy[1]), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 3)
                # when a person wear mask incorrectly
                else:
                    cv2.rectangle(img_org, (x + xy[0], y + xy[1]), (x2 + xy[0], y2 + xy[1]), (0, 255, 255), 4)
                    cv2.putText(img_org, "Incorrect mask", (x + xy[0], y - 10 + xy[1]), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 3)
            # when a person wears a mask correctly
            elif correctness == 1:
                cv2.rectangle(img_org, (x + xy[0], y + xy[1]), (x2 + xy[0], y2 + xy[1]), (0, 255, 0), 4)
                cv2.putText(img_org, "Correct mask", (x + xy[0], y - 10 + xy[1]), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
            

    except:
        print("error")
        pass
        
    newname = '/gdrive/My Drive/output/final_final/{0:05d}_test.jpg'.format(i)
    cv2.imwrite(newname,img_org)

00000: color name: navy
00001: color name: navy
00002: color name: purple
00003: color name: purple
00004: color name: purple
00005: color name: purple
00006: color name: purple
00007: color name: purple
00008: color name: purple
00009: color name: purple
error
00010: color name: purple
error
00011: color name: purple
00012: color name: purple
00013: color name: purple
00014: color name: navy
00015: color name: gray
00016: color name: gray
00017: color name: purple
00018: color name: maroon
00019: color name: maroon
00020: color name: gray
00021: color name: maroon
00022: color name: purple
00023: color name: purple
00024: color name: maroon
00025: color name: maroon
00026: color name: gray
00027: color name: maroon
00028: color name: purple
00029: color name: gray
00030: color name: purple
00031: color name: purple
00032: color name: purple
00033: color name: gray
00034: color name: gray
00035: color name: gray
00036: color name: gray
00037: color name: gray
00038: color name: gray
00

# 6. Make gif file with result images

In [ ]:
import imageio
import matplotlib.image as mpimg

In [ ]:
path = [f"/gdrive/My Drive/output/final_final/{i}" for i in os.listdir("/gdrive/My Drive/output/final_final/")]
paths = [ cv2.resize(np.array(Image.open(i)),dsize=(0,0),fx=0.5,fy=0.5,interpolation=cv2.INTER_LINEAR) for i in path]
imageio.mimsave('/gdrive/My Drive/output/final_final.gif', paths, fps=60)